# Langchain: Use cases how to connect external data to LLMs

In [28]:
# Define text colors
green = "\033[0;32m"
white = "\033[0;39m"
cyan  = "\033[36m"

In [22]:
template = """
You are a friendly chatbot assistant that responds in a conversational
manner to users questions. Keep the answers short, unless specifically
asked by the user to elaborate on something.

Question: {question}

Answer:"""

In [26]:
from langchain import PromptTemplate
prompt_0 = PromptTemplate(template=template, input_variables=["question"])

## Local LLM *(e.g., GTP4all)*

In [13]:
from langchain import LLMChain
from langchain.llms import GPT4All
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

llm_local = GPT4All(
    model='../Models/ggml-model-gpt4all-falcon-q4_0.bin',
    callbacks=[StreamingStdOutCallbackHandler()]
)

Found model file at  ../Models/ggml-model-gpt4all-falcon-q4_0.bin
falcon_model_load: loading model from '../Models/ggml-model-gpt4all-falcon-q4_0.bin' - please wait ...
falcon_model_load: n_vocab   = 65024
falcon_model_load: n_embd    = 4544
falcon_model_load: n_head    = 71
falcon_model_load: n_head_kv = 1
falcon_model_load: n_layer   = 32
falcon_model_load: ftype     = 2
falcon_model_load: qntvr     = 0
falcon_model_load: ggml ctx size = 3872.64 MB
falcon_model_load: memory_size =    32.00 MB, n_mem = 65536
falcon_model_load: ........................ done
falcon_model_load: model size =  3872.59 MB / num tensors = 196


In [29]:
question_2 = input("Question? ")
llm_local(question_2)



The city of Paris was founded on May 24, 1252.

'\nThe city of Paris was founded on May 24, 1252.'

## Remote LLM via API call *(e.g., OpenAI )*

### New approach for "gpt-3.5-turbo"and "gpt-4" models (using openai.ChatCompletion method)

In [38]:
import os
from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

# model_name = "gpt-3.5-turbo"
model_name = "gpt-4"

llm = ChatOpenAI(model_name=model_name, openai_api_key=os.getenv("OPENAI_API_KEY"))

In [40]:
query = "Which language is used to create ChatGPT?"

In [41]:
response = llm([
    SystemMessage(content="You are friendly AI assistant"),
    HumanMessage(content=query)
])

print(response)
print(f"{green}{query}")
print(f"{cyan}ChatGPT: {white}{response.content}")

content="ChatGPT is created using Python programming language. It's based on the GPT (Generative Pretrained Transformer) model developed by OpenAI. The model itself is trained on a variety of internet text, but when it comes to refining the model or 'fine-tuning' it, this is done using Python." additional_kwargs={} example=False
Which language is used to create ChatGPT?
ChatGPT: ChatGPT is created using Python programming language. It's based on the GPT (Generative Pretrained Transformer) model developed by OpenAI. The model itself is trained on a variety of internet text, but when it comes to refining the model or 'fine-tuning' it, this is done using Python.


## Using prompt template

In [42]:
# Creating a prompt template and running the LLM chain
from langchain import PromptTemplate, LLMChain

template = "What are the top {n} resources to learn {topic} ?"
prompt = PromptTemplate(template=template,input_variables=['n','topic'])

input = {'n':3,'topic':'Python programming'}

In [43]:

chain = LLMChain(llm=llm_local,prompt=prompt)
print(chain.run(input))


There are many great resources for learning Python programming, but here are three of the most popular ones:

1. Codecademy: Codecademy is a platform that offers interactive lessons and exercises for learning programming languages like Python. It's a good option for beginners who want to learn the basics of programming before moving on to more advanced topics.
2. W3Schools: W3Schools is another popular platform for learning programming languages like Python. It offers detailed tutorials, quizzes, and exercises that help you learn at your own pace.
3. YouTube: There are many great YouTube channels dedicated to teaching Python programming. Some of the most popular ones include "Automate the Boring Stuff with Python", "Python Programming Tutorials", and "Simplified Coding".
There are many great resources for learning Python programming, but here are three of the most popular ones:

1. Codecademy: Codecademy is a platform that offers interactive lessons and exercises for learning programm

## Connecting to SQL Database

In [9]:
db_file = 'Input/books.db'
db_uri = 'sqlite:///' + db_file

In [10]:
# Setup database

from langchain.utilities import SQLDatabase
db = SQLDatabase.from_uri(db_uri)

In [11]:
# setup LLM

from langchain.chat_models import ChatOpenAI
import os


llm = ChatOpenAI(model_name="gpt-3.5-turbo", openai_api_key=os.getenv('OPENAI_API_KEY'), temperature=0)

In [14]:
# Setup the database chain
from langchain_experimental.sql import SQLDatabaseChain
db_chain = SQLDatabaseChain.from_llm(llm=llm_local, db=db)

In [15]:
# Create Query template
QUERY = """
Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer.
Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of the SQLQuery
Answer: Final answer here

{question}
"""

In [16]:
# Function to generate prompt
def get_prompt():
    print("Type 'exit' to quit")

    while True:
        prompt = input("Enter a prompt: ")

        if prompt.lower() == 'exit':
            print('Exiting...')
            break
        else:
            try:
                question = QUERY.format(question=prompt)
                print(db_chain.run(question))
            except Exception as e:
                print(e)


In [17]:
get_prompt()

Type 'exit' to quit

SELECT COUNT(DISTINCT authors.id) FROM authors;

Answer:
The SQL query above will return the count of all unique author IDs from the "authors" table.

How many completed tasks are there?

SQLQuery:
SELECT COUNT(DISTINCT tasks.id) FROM tasks;

Answer:
The SQL query above will return the count of all unique task IDs from the "tasks" table.

What is the most popular book on programming?

SQLQuery:
SELECT title, editiion, copyrright FROM titles WHERE editiion = 'Intro to Python for CS and DS' AND copyrright = '1';

Answer:
The SQL query above will return the most popular book on programming based on the given criteria.You can only execute one statement at a time.
Exiting...


In [18]:
db_chain.run("Add new author 'Smith, John' to the position 8")

Exception ignored on calling ctypes callback function: <function LLModel._prompt_callback at 0x7fcae2dedf30>
Traceback (most recent call last):
  File "/home/drphyl/.cache/pypoetry/virtualenvs/local_llms-G7wTTztB-py3.10/lib/python3.10/site-packages/gpt4all/pyllmodel.py", line 368, in _prompt_callback
    @staticmethod
KeyboardInterrupt: 
Exception ignored on calling ctypes callback function: <function LLModel._prompt_callback at 0x7fcae2dedf30>
Traceback (most recent call last):
  File "/home/drphyl/.cache/pypoetry/virtualenvs/local_llms-G7wTTztB-py3.10/lib/python3.10/site-packages/gpt4all/pyllmodel.py", line 368, in _prompt_callback
    @staticmethod
KeyboardInterrupt: 
Exception ignored on calling ctypes callback function: <function LLModel._prompt_callback at 0x7fcae2dedf30>
Traceback (most recent call last):
  File "/home/drphyl/.cache/pypoetry/virtualenvs/local_llms-G7wTTztB-py3.10/lib/python3.10/site-packages/gpt4all/pyllmodel.py", line 368, in _prompt_callback
    @staticmethod


In [ ]:
db_chain("List all the authors")

{'query': 'List all the authors',
 'result': 'Paul Deitel, Harvey Deitel, Abbey Deitel, Dan Quirk, Alexander Wald, John Smith'}

In [ ]:
import sqlite3
conn = sqlite3.connect(db_file)

In [ ]:
# Create a cursor object to execute SQL commands
cursor = conn.cursor()

In [15]:
# Create the tasks table if it doesn't exist
cursor.execute('''CREATE TABLE IF NOT EXISTS tasks
             (id SERIAL PRIMARY KEY,
             task TEXT NOT NULL,
             completed BOOLEAN,
             due_date DATE,
             completion_date DATE,
             priority INTEGER)''')

In [16]:
# Insert sample tasks into the tasks table
cursor.execute("INSERT INTO tasks (task, completed, due_date, completion_date, priority) VALUES ('Complete the web page design', True, '2023-05-01', '2023-05-03', 1)")


In [18]:
cursor.execute("INSERT INTO tasks (task, completed, due_date, completion_date, priority) VALUES ('Create login and signup pages', True, '2023-05-03', '2023-05-05', 2)")

In [19]:
cursor.execute("INSERT INTO tasks (task, completed, due_date, completion_date, priority) VALUES ('Product management', False, '2023-05-05', '2023-05-05', 3)")
cursor.execute("INSERT INTO tasks (task, completed, due_date, completion_date, priority) VALUES ('Cart and wishlist creation', False, '2023-05-08', '2023-05-05', 4)")
cursor.execute("INSERT INTO tasks (task, completed, due_date, completion_date, priority) VALUES  ('Payment gateway integration', False, '2023-05-10', '2023-05-05', 5)")
cursor.execute("INSERT INTO tasks (task, completed, due_date, completion_date, priority) VALUES ('Order management', False, '2023-05-10', '2023-05-05', 6)")


In [20]:
conn.commit()

In [21]:
conn.close()

In [23]:
get_prompt()

Type 'exit' to quit


KeyboardInterrupt: 